In [ ]:
import numpy as np
import random
import scipy.sparse as sp
import torch
import os
import time
import pandas as pd
import pickle
import copy

In [ ]:
!pip install dgl

In [ ]:
!pip install dgl-cu101

     |████████████████████████████████| 36.2 MB 12.1 MB/s 


In [ ]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 12.2 MB/s 
     |████████████████████████████████| 596 kB 20.7 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
     |████████████████████████████████| 6.5 MB 35.7 MB/s 
     |████████████████████████████████| 895 kB 44.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
os.environ["DGLBACKEND"] = "pytorch"

In [ ]:
SEED = 42
os.environ['PYTHONHASHSEED']=str(SEED)
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'  # TF 2.1
random.seed(SEED)
#seed(SEED)
np.random.seed(SEED)

In [ ]:
import dgl
from dgl import save_graphs, load_graphs
from dgl.data import DGLDataset
import dgl.nn.pytorch as dglnn
from dgl.utils import expand_as_pair
import dgl.function as fn
import torch.nn as nn
from dgl.dataloading import GraphDataLoader
import torch.nn.functional as F
from torch import optim

from dgl.dataloading import GraphDataLoader
from torch.utils.data.sampler import SubsetRandomSampler

from transformers import GPT2Tokenizer, GPT2Model
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score  
from sklearn.metrics import roc_auc_score

Using backend: pytorch


In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')

comment = 'bye ! dont look come or think of comming back ! tosser'
encoded_input = tokenizer(comment, return_tensors='pt')
print("encoded_input", encoded_input)
output = model(**encoded_input)
outputseq = output.last_hidden_state[0]
words = comment.split()
print("words", len(words), len(comment))
print(outputseq.shape)

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

encoded_input {'input_ids': tensor([[16390,  5145, 17666,   804,  1282,   393,   892,   286,   725,   278,
           736,  5145, 12153,   263]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
words 12 54
torch.Size([14, 768])


In [ ]:
outputseq[8:10].shape

torch.Size([2, 768])

In [ ]:
inputs = tokenizer('bye ! dont look come or think of comming back ! tosser')
encoded_sequence = inputs["input_ids"]
decoded_sequence = tokenizer.decode(encoded_sequence)
decoded_sequence

'bye! dont look come or think of comming back! tosser'

In [ ]:
encoded_sequence

[16390, 5145, 17666, 804, 1282, 393, 892, 286, 725, 278, 736, 5145, 12153, 263]

In [ ]:
def subwords_to_merge(tokenized_sequence, sentence, verbose):
  for i in range(len(tokenized_sequence)) :
    while 'Ġ' in tokenized_sequence[i] :
      tokenized_sequence[i] = tokenized_sequence[i].replace('Ġ','')
  words = sentence.split(' ')
  if verbose :
    print("words", words)
    print("token", tokenized_sequence)
  i = 0
  j = 0
  cpt = 0
  n_words = len(words)
  list_subwords_to_merge = []

  while cpt != n_words :
    if tokenized_sequence[j] == words[i] :
      cpt = cpt + 1
      i = i + 1
      j = j + 1
    else :
      tmp_word = tokenized_sequence[j]
      tmp_merge = [j]
      while tmp_word != words[i] :
        j = j+1
        tmp_word = tmp_word + tokenized_sequence[j]
        tmp_merge.append(j)
      list_subwords_to_merge.append(tmp_merge)
      cpt = cpt + 1
      i = i + 1
      j = j + 1
  return list_subwords_to_merge


def get_embedding(comment, tokenizer, model, verbose):
  tokenized_sequence = tokenizer.tokenize(comment)
  subwords_indices = subwords_to_merge(tokenized_sequence, comment, verbose)
  encoded_input = tokenizer(comment, return_tensors='pt')
  output = model(**encoded_input)
  outputseq = output.last_hidden_state[0]

  final_embd = []
  i = 0

  while i != len(outputseq):
    inside = False
    for k in subwords_indices :
      first, last = k[0], k[-1]
      if i in range(first, last) :
        inside = True
        if (last+1)>len(outputseq):
          merge, _ = torch.max(outputseq[first::], 0)
        else:
          merge, _ = torch.max(outputseq[first:(last+1)], 0)
        final_embd.append(list(merge.detach().numpy()))
        i =  i + len(k)
        
    if inside == False:
      final_embd.append(list(outputseq[i].detach().numpy()))
      i = i + 1

  final_embd = torch.tensor(np.array(final_embd))
  return final_embd

In [ ]:
def sentence_process(sentence):
  comment = sentence.replace('\n', ' ')
  while '\"' in comment :
    comment = comment.replace('\"', '')
  while "\'" in comment :
    comment = comment.replace("\'", '')
  while ':' in comment :
    comment = comment.replace(':', '')
  while '.' in comment :
    comment = comment.replace('.', '')
  while '@' in comment :
    comment = comment.replace('@', '')
  while '+' in comment :
    comment = comment.replace('+', '')
  while '=' in comment:
    comment = comment.replace('=', '')
  while '&' in comment :
    comment = comment.replace('&', '')
  while ')' in comment or '(' in comment:
    comment = comment.replace(')', '').replace('(', '')
  while ',' in comment or ':' in comment or ';' in comment:
    comment = comment.replace(":", '').replace(',', '').replace(';', '')
  for c in comment :
    if c.isascii() == False :
      comment = comment.replace(c, '')
  n1 = comment.count('!')
  n2 = comment.count('?')
  for i in range(n1):
    comment = comment.replace('!', ' !')
  for i in range(n2):
    comment = comment.replace('?', ' ?')
  while '  ' in comment :
    comment = comment.replace('  ', ' ')
  return comment
  
class ToxicCommentDataset(DGLDataset):
    def __init__(self, save_dir):
        super().__init__(name='toxic', save_dir = save_dir)
        #self.save_dir = 

    def process(self):
        tic = time.time()
        p , q = 1, 2
        self.graphs = []
        self.labels = []

        train_df = pd.read_csv('/content/drive/My Drive/IASD_tmp/NLP/train.csv')
        tic = time.time()

        #delete comments that belongs to several classes
        idx_to_del = []
        for row in train_df.itertuples():
          if sum(row[3::])> 1 :
            idx_to_del.append(row[0])
          
        train_df = train_df.drop(idx_to_del)

        tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        model = GPT2Model.from_pretrained('gpt2')

        cpt = 0
        n_too_long = 0


        toxic = train_df[train_df['toxic'] == 0]
        obscene = toxic[toxic['obscene']==0]
        threat = obscene[obscene['threat']==0]
        insult = threat[threat['insult']==0]
        neutral = insult[insult['identity_hate']==0] #df with neutral comment

        toxic = train_df[train_df['toxic'] == 1]
        obscene = train_df[train_df['obscene'] == 1]
        threat = train_df[train_df['threat'] == 1]
        insult = train_df[train_df['insult'] == 1]
        identity_hate = train_df[train_df['identity_hate'] == 1]

        final_df = pd.concat([toxic, obscene, threat, insult, identity_hate, neutral]) #merge all the df together in the right order
        for row in final_df.itertuples():
      
          cpt = cpt + 1
          print("cpt:", cpt)
          
         
          if cpt in [1344, 1345, 1445, 1446, 1528, 1529, 3243, 3244, 3773, 3776, 11888] :
            continue
          if len(self.graphs) == 10000 or len(self.labels) == 10000:
            if len(self.graphs) == len(self.labels):
              break
            else : 
              print("Graphs and labels different shapes")
              return -1
          
          
          comment = sentence_process(row[2])
          words = comment.split(' ')
         
          while '' in words :
            words.remove('')
          
          for w in words :
            if w.isnumeric() and len(w)>=5 :
              words.remove(w)
          
          words = list(filter(lambda s: 'http' not in s, words))
            
          comment = " ".join(words)

          if len(words)>150 : #comments with more than 1024 words = cannot use with GPT2 or more than 996 : outliers comments
            print("Too long")
            n_too_long += 1
            continue


          #print("comment:", comment)
          if row[3] == 0 and row[4] == 0 and row[5] ==0 and row[6] ==0 and row[7] == 0 and row[8] == 0:
            self.labels.append(0)
          elif row[3] == 1 :
            self.labels.append(1)
          elif row[4] == 1 :
            self.labels.append(6)
          elif row[5] == 1 :
            self.labels.append(2)
          elif row[6] == 1 :
            self.labels.append(3)
          elif row[7] == 1 :
            self.labels.append(4)
          else: 
            self.labels.append(5)
      
          #Get embeddings for the sentence
          verbose = False
          outputseq = get_embedding(comment, tokenizer, model, verbose)
          
          #Isolate words
          words = comment.split(' ')

          assert len(words) == len(outputseq)
  
          #Fill the embedding dic for the comment
          dic_embeddings = {}
          for k, word in enumerate(words) :
            dic_embeddings[word] = outputseq[k].detach().numpy()
          
          dic_nodes_int = {}
          doubles = {}
          total_doubles = 0
          list_embeddings = []

          for word in words :
            if word in dic_nodes_int : ### cas de doublons
              dic_nodes_int[word] = [dic_nodes_int[word], len(dic_nodes_int)]
              doubles[word] = 0
              total_doubles = total_doubles + 1
            else :
              dic_nodes_int[word] = len(dic_nodes_int) + total_doubles

            list_embeddings.append(dic_embeddings[word])

          list_embeddings = np.array(list_embeddings)

          #### BOTTOM ####
          bottom_list_src = []
          bottom_list_dst = []

          for j, word in enumerate(words) : 

            neighbors = words[max((j-p),0):j] + words[j:(j+p+1)]
            neighbors_idx = [m for m in range(max((j-p), 0), max((j+p+1), len(neighbors)))]
            dic_nodes_int_neighbors = []

            for r, neigh in enumerate(neighbors):
              if neigh in doubles : 
                dic_nodes_int_neighbors.append(neighbors_idx[r])
              else :
                dic_nodes_int_neighbors.append(dic_nodes_int[neigh])

            bottom_list_src = bottom_list_src + [j for k in range(len(neighbors))] + dic_nodes_int_neighbors # Add edges in both directions
            bottom_list_dst = bottom_list_dst + dic_nodes_int_neighbors + [j for k in range(len(neighbors))] #

          #### MIDDLE ####
          middle_list_src = []
          middle_list_dst = []
          for j, word in enumerate(words) :

            neighbors = words[max((j-q),0):j] + words[j:(j+q+1)]
            neighbors_idx = [m for m in range(max((j-q), 0), max((j+q+1), len(neighbors)))]
            dic_nodes_int_neighbors = []

            for r, neigh in enumerate(neighbors):
              if neigh in doubles : 
                dic_nodes_int_neighbors.append(neighbors_idx[r])
              else :
                dic_nodes_int_neighbors.append(dic_nodes_int[neigh])

            middle_list_src = middle_list_src + [j for k in range(len(neighbors))] + dic_nodes_int_neighbors # Add edges in both directions
            middle_list_dst = middle_list_dst + dic_nodes_int_neighbors + [j for k in range(len(neighbors))] #

          #### TOP ####
          top_list_src = []
          top_list_dst = []
          for j, word in enumerate(words) :
            
            top_list_src = top_list_src + [j for k in range(len(words))] + [p for p in range(len(words))] # Add edges in both directions
            top_list_dst = top_list_dst + [p for p in range(len(words))] + [j for k in range(len(words))] #
          

          g = dgl.heterograph({ ('word', 'bottom', 'word'): (torch.tensor(bottom_list_src), torch.tensor(bottom_list_dst)), 
                                ('word', 'middle', 'word') : (torch.tensor(middle_list_src), torch.tensor(middle_list_src)), 
                                ('word', 'top', 'word'):(torch.tensor(top_list_src), torch.tensor(top_list_dst)) })
          
          
      
          features = torch.tensor(list_embeddings, dtype = torch.float32)
         
          g.ndata['word'] = features
          self.graphs.append(g)
        
        
        #Train/Val/Test/Split
        num_examples = len(self.graphs)
        num_train = int(0.80 * num_examples)
        num_val = int((num_examples-num_train)/2)

        self.graphs, self.labels = shuffle(self.graphs, self.labels, random_state = 0)
        train_graphs, train_labels = self.graphs[:num_train], torch.LongTensor(self.labels[:num_train])
        val_graphs, val_labels = self.graphs[num_train:(num_train+num_val)], torch.LongTensor(self.labels[num_train : (num_train+num_val)])
        test_graphs, test_labels = self.graphs[(num_train+num_val) : (num_train+2*num_val)], torch.LongTensor(self.labels[(num_train+num_val):(num_train+2*num_val)])

        # Convert the label list to tensor for saving.
        #self.labels = torch.LongTensor(self.labels)
        train_graph_path = ('/content/drive/My Drive/IASD_tmp/NLP/train_dgl_graph.bin')
        val_graph_path = ('/content/drive/My Drive/IASD_tmp/NLP/val_dgl_graph.bin')
        test_graph_path = ('/content/drive/My Drive/IASD_tmp/NLP/test_dgl_graph.bin')
        
        save_graphs(train_graph_path, train_graphs, {'labels': train_labels})
        save_graphs(val_graph_path, val_graphs, {'labels': val_labels})
        save_graphs(test_graph_path, test_graphs, {'labels': test_labels})

    def __getitem__(self, i):
        return self.graphs[i], self.labels[i]

    def __len__(self):
        return len(self.graphs)

    def has_cache(self):
        if os.path.exists(self.save_dir):
          self.graphs, label_dict = load_graphs(self.save_dir)
          self.labels = label_dict['labels']
          return True

train_dataset = ToxicCommentDataset(save_dir = '/content/drive/My Drive/IASD_tmp/NLP/train_dgl_graph.bin')
val_dataset = ToxicCommentDataset(save_dir = '/content/drive/My Drive/IASD_tmp/NLP/val_dgl_graph.bin')
test_dataset = ToxicCommentDataset(save_dir = '/content/drive/My Drive/IASD_tmp/NLP/val_dgl_graph.bin')

In [ ]:
class MultiHeadGATLayer(nn.Module):
    def __init__(self, in_feats, out_feats, num_heads):
        super(MultiHeadGATLayer, self).__init__()
        self.heads = nn.ModuleList()
        for i in range(num_heads):
            self.heads.append(dgl.nn.pytorch.conv.GATConv(in_feats, in_feats, num_heads, allow_zero_in_degree=True))

    def forward(self, graph, feat):
    
        head_outs = [attn_head(graph, feat) for attn_head in self.heads]
        return torch.mean(torch.stack(head_outs))


class MultiDotGATLayer(nn.Module):
    def __init__(self, in_feats, out_feats, h_dot, num_heads):
        super(MultiDotGATLayer, self).__init__()
        self.heads = nn.ModuleList()
        for i in range(h_dot):
            self.heads.append(dglnn.conv.DotGatConv(in_feats, out_feats, num_heads))

    def forward(self, graph, feat):
        head_outs = [attn_head(graph, feat) for attn_head in self.heads]
        concat_feat = torch.stack(head_outs)
        mean = torch.mean(concat_feat, dim = 0)
        mean = torch.mean(mean, dim = 1)
        return mean

class BottomConv(nn.Module):
    def __init__(self,
                 in_feats,
                 out_feats,
                 beta,
                 activation=None):
        super(BottomConv, self).__init__()

        self.gin = dgl.nn.pytorch.conv.GINConv(None, 'mean', init_eps=beta, learn_eps=True)
        self.activation = activation
        self.beta = beta
        self.linear = nn.Linear(in_feats, out_feats)

    def forward(self, graph, feat):
        #graph.update_all(fn.copy_u('h', 'm'), fn.mean('m', 'h_mean'))
        #h_mean = graph.dstdata['h_mean']
        #print("h_mean:", h_mean)
        #rst = torch.mul(h_mean, 1-self.beta) + torch.mul(feat, self.beta)
        feat = feat[0]
        h = self.gin(graph, feat)
        h = self.linear(h)
        h = self.activation(h)
        return h


class MiddleConv(nn.Module):
  def __init__(self,
                 in_feats,
                 out_feats,
                 num_heads,
                 gamma,
                 activation=None):
        super(MiddleConv, self).__init__()
        
        self.gamma = gamma
        self.multigat = MultiHeadGATLayer(in_feats, in_feats, num_heads)
        self.linear = nn.Linear(in_feats, out_feats)
        self.activation = self.activation = activation

  def forward(self, graph, feat):
    feat = feat[0]
    h = self.multigat(graph, feat)
    h = torch.mul(h, 1-self.gamma) + torch.mul(feat, self.gamma)
    h = self.linear(h)
    h = self.activation(h)
    return h

class TopConv(nn.Module):
  def __init__(self,
               in_feats,
               out_feats, 
               h_dot, 
               num_heads, 
               activation = None):
    super(TopConv, self).__init__()
    self.multidotgat = MultiDotGATLayer(in_feats, out_feats, h_dot, num_heads)
    self.linear = nn.Linear(out_feats, h_dot * out_feats, bias=False)
    self.activation = activation
  
  def forward(self, graph, feat):
    feat = feat[0]
    h = self.multidotgat(graph, feat)
    h = self.linear(h)
    h = self.activation(h)
    return h
          
class MLGNN(nn.Module):
    def __init__(self, d0, d1, d2, d3, h_dot, beta, gamma, num_heads, c):
        super().__init__()
        self.d3 = d3
        self.h_dot = h_dot
        self.c = c

        self.conv_bottom = dglnn.HeteroGraphConv({'bottom' : BottomConv(d0, d1, beta, nn.ReLU())}, aggregate='sum')
        self.conv_middle = dglnn.HeteroGraphConv({'middle': MiddleConv(d1, d2, num_heads, gamma, nn.ReLU())}, aggregate='sum')
        self.conv_top = dglnn.HeteroGraphConv({'top': TopConv(d2, d3, self.h_dot, num_heads, nn.ReLU())}, aggregate='sum')
        self.linear = nn.Linear(self.h_dot * self.d3, self.c)
        #self.classify = nn.Softmax(dim=0)

    def forward(self, g):
        bottom_g = g.edge_type_subgraph(['bottom'])
        middle_g = g.edge_type_subgraph(['middle'])
        top_g = g.edge_type_subgraph(['top'])

        h = g.ndata
        h = self.conv_bottom(bottom_g, h)
        h = self.conv_middle(middle_g, h)
        h = self.conv_top(top_g, h)
        g.ndata['word'] = h['word']

        h = dgl.readout_nodes(g, 'word', op='max')
        h = self.linear(h)
        #h = self.classify(h)
        return h

In [ ]:
def evaluate(testloader, model):
  num_correct = 0
  num_tests = 0
  for batched_graph, labels in testloader:
    batched_graph = batched_graph.to(device)
    labels = labels.to(device)
    with torch.no_grad():
      pred = model(batched_graph)
    num_correct += (pred.argmax(1) == labels).sum().item()
    num_tests += len(labels)

  return num_correct / num_tests

In [ ]:
def predict(testloader, model):
  preds = []
  true_labels = []
  for batched_graph, labels in testloader:
    true_labels.append(labels)
    batched_graph = batched_graph.to(device)
    labels = labels.to(device)
    with torch.no_grad():
      pred = model(batched_graph)
      preds.append(preds)
  preds = torch.cat(preds, dim=0)
  true_labels = torch.cat(true_labels, dim=0)
  return preds, true_labels

In [ ]:
d0 = 768
c = 6 ###TODO, attention, reprendre en fct du nouveau dataset

In [ ]:
n_experiments = 1

hyperparams_search = [{'d1':816, 'd2':912, 'd3':1024, 'h_dot':2, 'num_heads':2, 'beta':1/2, 'gamma':1/2, 'batch_size':1024, 'n_epochs':20, 'verbose':True, 'lr':0.001},
                      {'d1':816, 'd2':912, 'd3':700, 'h_dot':2, 'num_heads':2, 'beta':1/2, 'gamma':1/2, 'batch_size':1024, 'n_epochs':20, 'verbose':True, 'lr':0.001},
                      {'d1':816, 'd2':912, 'd3':1024, 'h_dot':2, 'num_heads':2, 'beta':1/3, 'gamma':1/3, 'batch_size':1024, 'n_epochs':20, 'verbose':True, 'lr':0.001}]
seeds = [i for i in range(n_experiments)]

validation_acc = []
test_acc = []

validation_f1 = []
test_f1 = []

validation_auc = []
test_auc = []

for args in hyperparams_search :
  val_seed_acc = []
  test_seed_acc = []

  val_seed_f1 = []
  test_seed_f1 = []

  val_seed_auc = []
  test_seed_auc = []

  for s in seeds :

    torch.manual_seed(s)
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    
    train_dataloader = GraphDataLoader(train_dataset,batch_size=args['batch_size'],num_workers=0, drop_last=False)
    val_dataloader = GraphDataLoader(val_dataset, batch_size = args['batch_size'],  num_workers=0, drop_last = False)
    test_dataloader = GraphDataLoader(test_dataset, batch_size=args['batch_size'],  num_workers=0, drop_last=False)

    labels_weights = train_dataset[:][1].numpy()
    #class_weights = torch.from_numpy(compute_class_weight('balanced', classes = np.unique(labels_weights), y=labels_weights))
    #class_weights = torch.cat((class_weights[0:2], torch.tensor([0.001]), class_weights[2::]))
    #class_weights = class_weights.to(dtype = torch.float32).to(device)

    model = MLGNN(d0, args['d1'],args['d2'], args['d3'], args['h_dot'], args['h_dot'], args['gamma'], args['num_heads'], c)
    model = model.to(device)


    optimizer = optim.Adam(model.parameters(), lr=args['lr'])

    epoch_losses = []
    for epoch in range(args['n_epochs']):
      epoch_loss = 0
      iter = 0
      for batched_graph, labels in train_dataloader:
          batched_graph = batched_graph.to(device)
          labels = labels.to(device)
          logits = model(batched_graph)
          loss = F.cross_entropy(logits, labels) #weight = class_weights)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          epoch_loss += loss.detach().item()
          iter += 1

      epoch_loss /= (iter + 1)
      if args['verbose']:
        print('Epoch {}, loss {:.4f}'.format(epoch, epoch_loss))
      epoch_losses.append(epoch_loss)
    
    val_seed_acc.append(evaluate(val_dataloader, model))
    test_seed_acc.append(evaluate(test_dataloader, model))

    val_preds, val_labels = predict(val_dataloader, model)
    test_preds, test_labels = predict(test_dataloader, model)

    val_seed_f1.append(f1_score(val_labels, val_preds))   ######TODO voir si fct en proba
    test_seed_f1.append(f1_score(test_labels, test_preds))

    val_seed_auc.append(roc_auc_score(val_labels, val_preds, multi_class="ovo", average="weighted"))
    test_seed_auc.append(roc_auc_score(test_labels, test_preds, multi_class="ovo", average="weighted"))



  #Mesure incertitude en fct seed  ####TODO ajouter std
  validation_acc.append(np.mean(val_seed_acc))  
  test_acc.append(np.mean(test_seed_acc))

  validation_f1.append(np.mean(val_seed_f1)) 
  test_f1.append(np.mean(test_seed_f1))

  validation_auc.append(np.mean(val_seed_auc))
  test_auc.append(np.mean(test_seed_auc))

#Mesure incertitude en fct split
print("Mean val acc :", np.mean(validation_acc), "+-", np.std(validation_acc))
print("F1-score val acc :", np.mean(validation_f1), "+-", np.std(validation_f1))

print("Mean test acc :", np.mean(test_acc), "+-", np.std(test_acc))
print("F1-score test acc :", np.mean(test_f1), "+-", np.std(test_f1))

Epoch 0, loss 1.1812
Epoch 1, loss 0.8841
Epoch 2, loss 0.8449
Epoch 3, loss 0.8325
Epoch 4, loss 0.8235
Epoch 5, loss 0.7914
Epoch 6, loss 0.7219
Epoch 7, loss 0.6572
Epoch 8, loss 0.7233
Epoch 9, loss 0.7046
Epoch 10, loss 0.6208
Epoch 11, loss 0.5701
Epoch 12, loss 0.5510
Epoch 13, loss 0.5250
Epoch 14, loss 0.6132
Epoch 15, loss 0.5981
Epoch 16, loss 0.5535
Epoch 17, loss 0.5400
Epoch 18, loss 0.5472
Epoch 19, loss 0.5143
val_seed_vals: [0.82]
test_seed_vals: [0.82]
Epoch 0, loss 1.1747
Epoch 1, loss 0.8862
Epoch 2, loss 0.8527
Epoch 3, loss 0.8456
Epoch 4, loss 0.8419
Epoch 5, loss 0.8288
Epoch 6, loss 0.7980
Epoch 7, loss 0.7157
Epoch 8, loss 0.6995
Epoch 9, loss 0.6246
Epoch 10, loss 0.6271
Epoch 11, loss 0.6098
Epoch 12, loss 0.5967
Epoch 13, loss 0.5688
Epoch 14, loss 0.5447
Epoch 15, loss 0.5439
Epoch 16, loss 0.5684
Epoch 17, loss 0.5353
Epoch 18, loss 0.5243
Epoch 19, loss 0.5197
val_seed_vals: [0.822]
test_seed_vals: [0.822]
Epoch 0, loss 1.1763
Epoch 1, loss 0.8960
Epoch 

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, preds)
  AUC.append(auc(fpr, tpr))

In [ ]:
from sklearn.metrics import roc_auc_score
weighted_roc_auc_ovo = roc_auc_score([0,1,2], , multi_class="ovo", average="weighted")
weighted_roc_auc_ovo

AxisError: ignored